## Importación de librerías

In [3]:
import pandas as pd
import os
import glob
import csv
import json

## Configuración de variables globales

In [4]:
with open('setup.json', 'r') as f:
    SETUP_JSON = json.load(f)
DATASETS_PATH = SETUP_JSON['datasets_path'] # Path to the datasets,
DATASETS_FOLDER = os.path.join(os.getcwd(), DATASETS_PATH) # Folder containing the datasets,
DATASETS = glob.glob(os.path.join(DATASETS_FOLDER, '*.csv')) # List of datasets
OUTPUT_CSV = SETUP_JSON['output_csv'] # Output CSV file
OUTPUT_PARQUET = SETUP_JSON['output_parquet'] # Output CSV file

## Análisis de los datasets

In [ ]:
# Obtener todas las columnas únicas y sus tipos
all_columns_dtypes = {}
for dataset in DATASETS:
    df_dtypes = pd.read_csv(dataset, nrows=1).dtypes
    df_dtypes.pop(dataset.columns[-1]) # Eliminar la columna de la etiqueta
    all_columns_dtypes.update(df_dtypes.to_dict())

print(f"Todas las columnas únicas: {len(all_columns_dtypes)}")
for column in all_columns_dtypes:
    print(f"Columna: {column}, Tipo: {all_columns_dtypes[column]}")

Todas las columnas únicas: 84
Columna: Dst Port, Tipo: int64
Columna: Protocol, Tipo: int64
Columna: Timestamp, Tipo: object
Columna: Flow Duration, Tipo: int64
Columna: Tot Fwd Pkts, Tipo: int64
Columna: Tot Bwd Pkts, Tipo: int64
Columna: TotLen Fwd Pkts, Tipo: int64
Columna: TotLen Bwd Pkts, Tipo: int64
Columna: Fwd Pkt Len Max, Tipo: int64
Columna: Fwd Pkt Len Min, Tipo: int64
Columna: Fwd Pkt Len Mean, Tipo: float64
Columna: Fwd Pkt Len Std, Tipo: float64
Columna: Bwd Pkt Len Max, Tipo: int64
Columna: Bwd Pkt Len Min, Tipo: int64
Columna: Bwd Pkt Len Mean, Tipo: int64
Columna: Bwd Pkt Len Std, Tipo: float64
Columna: Flow Byts/s, Tipo: float64
Columna: Flow Pkts/s, Tipo: float64
Columna: Flow IAT Mean, Tipo: float64
Columna: Flow IAT Std, Tipo: float64
Columna: Flow IAT Max, Tipo: int64
Columna: Flow IAT Min, Tipo: int64
Columna: Fwd IAT Tot, Tipo: int64
Columna: Fwd IAT Mean, Tipo: float64
Columna: Fwd IAT Std, Tipo: float64
Columna: Fwd IAT Max, Tipo: int64
Columna: Fwd IAT Min, T

## Combinar datasets

Unimos todos los archivos CSV y guardamos el dataset en formato CSV (para analizarlo con Tableau) y en formato Parquet (para su tratamiento con Python).

In [ ]:
# Añadir columnas faltantes
def add_missing_columns(filename, df):
    print(f"Procesando {filename}: {df.shape}")
    for col in all_columns_dtypes.keys():
        if col not in df.columns:
            df.insert(loc=df.shape[1] - 1, column=col, value=pd.NA)
    return df

In [ ]:
# Combinar los datasets
df = pd.DataFrame()
first_df = add_missing_columns(DATASETS[0], pd.read_csv(DATASETS[0], dtype=str))
df = pd.concat([df, first_df], ignore_index=True, sort=False)
for dataset in DATASETS[1:]:
    next_df = add_missing_columns(dataset, pd.read_csv(dataset, dtype=str))
    df = pd.concat([df, next_df], ignore_index=True, sort=False)

Procesando dataframe: (1048575, 80)
Procesando dataframe: (1048575, 80)
Procesando dataframe: (1048575, 80)
Procesando dataframe: (7948748, 84)
Procesando dataframe: (1048575, 80)
Procesando dataframe: (1048575, 80)
Procesando dataframe: (1048575, 80)
Procesando dataframe: (613104, 80)
Procesando dataframe: (331125, 80)
Procesando dataframe: (1048575, 80)


In [10]:
df.shape

(16233002, 84)

In [11]:
df.dtypes.unique()

array([dtype('O')], dtype=object)

In [ ]:
df.info()  # Display DataFrame information

In [ ]:
df.head()

## Exportación de archivos JSON

In [ ]:
# Convertir los tipos a strings
serializable_dtypes = {col: str(dtype) for col, dtype in all_columns_dtypes.items()}

# Guardar en JSON
with open('dtypes.json', 'w') as f:
    json.dump(serializable_dtypes, f, indent=4)

In [ ]:
# Obtener las etiquetas únicas
labels = {}
for label, i in enumerate(df[-1].unique()):
    labels[label] = i
labels.pop(df.columns[-1])
print(f"Etiquetas únicas: {labels}")

# Guardar etiquetas en JSON
with open('labels.json', 'w') as f:
    json.dump(labels, f, indent=4)

## Exportación del dataframe en CSV y Parquet

In [18]:
# Guardar en CSV
df.to_csv(OUTPUT_CSV, index=False)
print(f"Archivo CSV guardado como {OUTPUT_CSV}")

# Guardar en Parquet
df.to_parquet(OUTPUT_PARQUET, index=False)
print(f"Archivo Parquet guardado como {OUTPUT_PARQUET}")

Archivo CSV guardado como merged_output.csv
Archivo Parquet guardado como merged_output.parquet


In [87]:

def find_datasets_with_same_header():
    headers = {}
    same_files = {}

    for file in DATASETS:
        with open(file, newline='', encoding='utf-8') as f:
            header = tuple(next(csv.reader(f), None))
            if header:
                if header in headers:
                    headers[header].append("datasets\\"+file)
                else:
                    headers[header] = ["datasets\\"+file]
    print("Archivos con headers distintos:", tuple(headers.values())[1])
    return tuple(headers.values())[0]

In [88]:
csv_files = find_datasets_with_same_header()
print("Archivos con el mismo encabezado:", csv_files)

Archivos con headers distintos: ['datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traffic-network-classify\\datasets\\02-20-2018.csv']
Archivos con el mismo encabezado: ['datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traffic-network-classify\\datasets\\02-14-2018.csv', 'datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traffic-network-classify\\datasets\\02-15-2018.csv', 'datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traffic-network-classify\\datasets\\02-16-2018.csv', 'datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traffic-network-classify\\datasets\\02-21-2018.csv', 'datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traffic-network-classify\\datasets\\02-22-2018.csv', 'datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traffic-network-classify\\datasets\\02-23-2018.csv', 'datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traffic-network-classify\\datasets\\02-28-2018.csv', 'datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traffic-network-classify\\datasets\\03-01-2018.csv', 'datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traff